# Libraries

In [1]:
import networkx as nx
import queue

# Functions

## Logic Gates

In [2]:
def OR(k):
    A,B = k
    if A == None or B == None:
        return None
    return A|B

def AND(k):
    A,B = k
    if A == None or B == None:
        return None
    return A & B;

def XOR(k):
    A,B = k
    if A == None or B == None:
        return None
    return A ^ B

def NOT(A):
    if type(A) == list:
        A = A[0] 
    if A == None:
        return None
    return ~A+2	

All the functions for gate operations have been defined above.

## Netlist Coversion

In [3]:
def NetlistConvert(file_path):
    with open(file_path, 'r') as file:
            netlist = file.readlines()
    net =[] 
    t = 0
    for line in netlist:
            linesplit = line.split("#")[0].split('\n')[0].split('  ')
            net.append(linesplit[0])
    return net

This function converts each line of the netlist into a string and appends it into a list giving a list of all lines in the netlist

## NxConvert

In [4]:
def NxConvert(netlist):
        ele,pri = set(),set()
        a = {}
        b = []
        for line in netlist:
            split_line = line.split()
            if len(split_line ) == 5:
                gate_id, gate_type, input1, input2, output = split_line
                ele.add(output)
                pri.add(input1)
                pri.add(input2)
                a.update({output : gate_type})
                b.append((input1,output))
                b.append((input2,output))
            
            if len(split_line)== 4:
                gate_id, gate_type, input1, output = split_line
                ele.add(output)
                pri.add(input1)
                a.update({output : gate_type})
                b.append((input1,output))
            
            primary = pri-ele
            for k in primary:
                a.update({k : "PI"})
        return a,b

This `NxConvert` function creates a dict of the output and corresponding gate and another list of all the edges of the network.

## Input Reading

In [5]:
def Input(file_path):
    with open(file_path, 'r') as file:
            inp = file.readlines()
    val = inp[0].split()
    dic = {k : [] for k in val}
    inp.pop(0)
    for line in inp:
        l = line.split()
        for i in range(len(l)):
            dic[val[i]].append(int(l[i]))
    return dic


This function is used to read the *.input* files and covert them into a dictionaries of lists

## Evaluation

In [6]:
def Evaluate(gate,k):
        if gate == 'nand2':
            value = NOT(AND(k))
        elif gate == 'or2':
            value = OR(k)
        elif gate == 'and2':
            value = AND(k)
        elif gate == 'nor2':
            value = NOT(OR(k))
        elif gate == 'inv':
            value = NOT(k)
        elif gate == 'xor2':
            value = XOR(k)
        elif gate == 'xnor2':
            value = NOT(XOR(k))
        else:
            if type(k) == list:
                k = k[0]
            value = k
        return value

This function is used for evaluation of the Logic Gates

# Topologically ordered evaluation

In [7]:
def TopCalc(a,b,val):
    g = nx.DiGraph()
    g.add_edges_from(b)
    nx.set_node_attributes(g,a,name="Gate")
    top = list(nx.topological_sort(g))

    for n in top:
        pred =  list(g.predecessors(n))
        try:
            k = (val[pred[0]],val[pred[1]])
        except IndexError:
            try:
                k = val[pred[0]]
            except IndexError:
                continue
        ans = Evaluate(a[n],k)
        val.update({n : ans})
    return val

This function creates a topologically ordered network of the netlist and evaluates the values of the outputs based on this and returns a dict of all the values

In [8]:
def FinalCalc(filename1,filename2):
    net = NetlistConvert(filename1)
    a,b = NxConvert(net)
    dic = Input(filename2)
    out = []
    for key, value in dic.items():
        k = len(value)
    for i in range(k):
        val = {key: value[i] for key, value in dic.items()}
        try:
            out.append(TopCalc(a,b,val))
        except:
            print(f"For {filename1}, The system contains a cycle. Out of the domain of this code")
            return None    
    return out

This fuction is a combination of all the previous function where it creates a list of dicts for every row of outputs

In [9]:
def Topological(filename1,filename2,filename3):
    answer = FinalCalc(filename1,filename2)
    ans = []
    if answer != None:
        for i in range(len(answer)):
            sorted_keys = sorted(answer[4].keys())
            k = {key : answer[i][key] for key in sorted_keys}
            ans.append(k)
        with open(filename3 , "w") as f:
            for t in ans[0]:
                f.write(str(t)+' ')
            f.write('\n')
            for i in range(len(ans)):
                for t in ans[i]:
                    f.write(str(ans[i][t])+"  ")
                f.write('\n')
    return ans


This function takes the lidt of dicts in previous functions, sorts the keys of dicts in alphabetical order and writes the output into a chosen file 

# Event-driven evaluation

In [10]:
def Queue(file1,file2):
    net = NetlistConvert(file1)
    a,b = NxConvert(net)
    states = {key: None for key in a}
    ans = []
    inp = Input(file2)

    g = nx.DiGraph()
    g.add_edges_from(b)
    nx.set_node_attributes(g,a,name="Gate")

    q = queue.Queue()
    for key, value in inp.items():
        k = len(value)
    for j in range(k):
        val = {key: value[j] for key, value in inp.items()}
        if j == 0:
            for i in val:
                q.put(i)
            while q.empty() == False:
                qout = q.get()
                pre = list(g.predecessors(qout))
                if len(pre) == 0:
                    states[qout] = val[qout]
                else:
                    k = [states[i] for i in pre]
                    if len(k) == 1 and k[0] != None:
                        k = int(k[0])
                    states[qout] = Evaluate(a[qout],k)
                succ = list(g.successors(qout))
                for i in succ:
                    q.put(i)

        else:
            for i in val:
                if val[i] != states[i]:
                    q.put(i)
            while q.empty() == False:
                qout = q.get()
                pre = list(g.predecessors(qout))
                if len(pre) == 0:
                    states[qout] = val[qout]
                else:
                    k = [states[i] for i in pre]
                    states[qout] = Evaluate(a[qout],k)
                succ = list(g.successors(qout))
                for i in succ:
                    q.put(i)
        sorted_keys = sorted(states.keys())
        k = {key : states[key] for key in sorted_keys}
        ans.append(k)
    return ans


- This function carries out a event based output simulation where first the primary inputs are added to the queue and then as each element is popped out of the queue its successors are added to the queueand output is evaluated.
- For the changing inputs, only primary inputs whose values have been changed have been added and the same idea is repeated

In [11]:
def QueuePrint(filename1,filename2,filename3):
    ans = Queue(filename1,filename2)
    if ans != None:
        with open(filename3 , "w") as f:
            for t in ans[0]:
                f.write(str(t)+' ')
            f.write('\n')
            for i in range(len(ans)):
                for t in ans[i]:
                    f.write(str(ans[i][t])+"  ")
                f.write('\n')
    return ans
b = QueuePrint('c8.net','c8.inputs','c8q.out')


# Outputs

## Topological

In [32]:
a = Topological('c17.net','c17.inputs','c17.out')
b = Topological('c8.net','c8.inputs','c8.out')
c = Topological('parity.net','parity.inputs','parity.out')
d = Topological('c17_1.net','c17.inputs','c17.out')

For c17_1.net, The system contains a cycle. Out of the domain of this code


## Event Driven

In [12]:
a = QueuePrint('c432.net','c432.inputs','c432q.txt')

In [13]:
%timeit QueuePrint('c432.net','c432.inputs','c432q.out')

39 s ± 599 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- **Topologically ordered evaluation** is a more straightforward and predictable approach. We can estimate the time taken by it even if we change a few lines in the netlist so as to complicate the DiGraph. It iterates over all the nodes every single time.

- For an **Event based evaluation**, it is not as straightforward, it iterates over the successors of each input and its successors and the successors of its successors. It can be efficient in some cases but can also be more complex and time- taking in others.   

- As we can see above, time taken by Topologically ordered evaluation is smaller than that of Event based evaluation. This I believe is because there are not enough nodes in place for the event based evaluation to actually cause a significant efficiency increase.    

- If there are about 100 primary inputs then I believe in the evaluation of the time changing input, the time for topological evaluation will shoot up whereas time for event based evaluation would be comparable